# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
housing_df = pd.read_csv('cal_housing_clean.csv')
housing_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [3]:
housing_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [4]:
x_data = housing_df.drop(['medianHouseValue'], axis=1)
x_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [5]:
y_data = housing_df['medianHouseValue']
y_data[:5]

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: medianHouseValue, dtype: float64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.3,
    random_state=101
)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [9]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [10]:
X_train_new = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_train_new.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [11]:
X_test_new = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)
X_test_new.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
8816,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
7175,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
16714,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
14491,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [12]:
housing_df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [13]:
housing_median_age = tf.feature_column.numeric_column('housingMedianAge')
total_rooms = tf.feature_column.numeric_column('totalRooms')
total_bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
median_income = tf.feature_column.numeric_column('medianIncome')

In [14]:
feat_cols = [
    housing_median_age,
    total_rooms,
    total_bedrooms,
    population,
    households,
    median_income
]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [15]:
input_func = tf.estimator.inputs.pandas_input_fn(
    X_train_new,
    y_train,
    num_epochs=1000,
    batch_size=10,
    shuffle=True
)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [16]:
model = tf.estimator.DNNRegressor(
    hidden_units=[6, 6, 6],
    feature_columns=feat_cols
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpgy7lwew5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1042f6198>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [17]:
model.train(input_fn=input_func, steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpgy7lwew5/model.ckpt.
INFO:tensorflow:loss = 679442640000.0, step = 1
INFO:tensorflow:global_step/sec: 633.196
INFO:tensorflow:loss = 953201070000.0, step = 101 (0.158 sec)
INFO:tensorflow:global_step/sec: 752.587
INFO:tensorflow:loss = 839139700000.0, step = 201 (0.133 sec)
INFO:tensorflow:global_step/sec: 723.368
INFO:tensorflow:loss = 359501630000.0, step = 301 (0.138 sec)
INFO:tensorflow:global_step/sec: 724.072
INFO:tensorflow:loss = 523835930000.0, step = 401 (0.138 sec)
INFO:tensorflow:global_step/sec: 701.114
INFO:tensorflow:loss = 803575560000.0, step = 501 (0.143 sec)
INFO:tensorflow:global_step/sec: 681.742
INFO:tensorflow:loss = 268464950000.0, step = 601 (0.147 sec)
INFO:tensorflow:global_step/sec: 648.815
INFO:tensorflow:loss = 353747760000.0, step = 701 (0.154 sec)
INFO:tensorflow:global_step/sec: 719.234
INFO:tensorflow:loss = 3406

INFO:tensorflow:loss = 29080758000.0, step = 7801 (0.156 sec)
INFO:tensorflow:global_step/sec: 455.843
INFO:tensorflow:loss = 54439010000.0, step = 7901 (0.219 sec)
INFO:tensorflow:global_step/sec: 497.231
INFO:tensorflow:loss = 140395400000.0, step = 8001 (0.201 sec)
INFO:tensorflow:global_step/sec: 497.026
INFO:tensorflow:loss = 19783827000.0, step = 8101 (0.202 sec)
INFO:tensorflow:global_step/sec: 542.293
INFO:tensorflow:loss = 71807630000.0, step = 8201 (0.185 sec)
INFO:tensorflow:global_step/sec: 525.823
INFO:tensorflow:loss = 48819925000.0, step = 8301 (0.189 sec)
INFO:tensorflow:global_step/sec: 582.734
INFO:tensorflow:loss = 80751665000.0, step = 8401 (0.172 sec)
INFO:tensorflow:global_step/sec: 565.048
INFO:tensorflow:loss = 163757410000.0, step = 8501 (0.177 sec)
INFO:tensorflow:global_step/sec: 659.435
INFO:tensorflow:loss = 224644760000.0, step = 8601 (0.151 sec)
INFO:tensorflow:global_step/sec: 720.411
INFO:tensorflow:loss = 133792326000.0, step = 8701 (0.139 sec)
INFO:te

INFO:tensorflow:loss = 154906020000.0, step = 15701 (0.148 sec)
INFO:tensorflow:global_step/sec: 657.908
INFO:tensorflow:loss = 209282860000.0, step = 15801 (0.151 sec)
INFO:tensorflow:global_step/sec: 786.552
INFO:tensorflow:loss = 152378820000.0, step = 15901 (0.127 sec)
INFO:tensorflow:global_step/sec: 830.504
INFO:tensorflow:loss = 137684060000.0, step = 16001 (0.120 sec)
INFO:tensorflow:global_step/sec: 785.996
INFO:tensorflow:loss = 85569340000.0, step = 16101 (0.127 sec)
INFO:tensorflow:global_step/sec: 677.902
INFO:tensorflow:loss = 76677150000.0, step = 16201 (0.148 sec)
INFO:tensorflow:global_step/sec: 757.092
INFO:tensorflow:loss = 155940420000.0, step = 16301 (0.132 sec)
INFO:tensorflow:global_step/sec: 843.151
INFO:tensorflow:loss = 52866200000.0, step = 16401 (0.119 sec)
INFO:tensorflow:global_step/sec: 716.066
INFO:tensorflow:loss = 34707770000.0, step = 16501 (0.140 sec)
INFO:tensorflow:global_step/sec: 735.905
INFO:tensorflow:loss = 190835410000.0, step = 16601 (0.136 

INFO:tensorflow:global_step/sec: 539.014
INFO:tensorflow:loss = 37407430000.0, step = 23601 (0.184 sec)
INFO:tensorflow:global_step/sec: 522.739
INFO:tensorflow:loss = 165355390000.0, step = 23701 (0.190 sec)
INFO:tensorflow:global_step/sec: 525.119
INFO:tensorflow:loss = 140631980000.0, step = 23801 (0.189 sec)
INFO:tensorflow:global_step/sec: 662.323
INFO:tensorflow:loss = 47297320000.0, step = 23901 (0.152 sec)
INFO:tensorflow:global_step/sec: 678.753
INFO:tensorflow:loss = 84556690000.0, step = 24001 (0.148 sec)
INFO:tensorflow:global_step/sec: 602.388
INFO:tensorflow:loss = 84116770000.0, step = 24101 (0.166 sec)
INFO:tensorflow:global_step/sec: 622.732
INFO:tensorflow:loss = 157354410000.0, step = 24201 (0.159 sec)
INFO:tensorflow:global_step/sec: 646.407
INFO:tensorflow:loss = 77012050000.0, step = 24301 (0.155 sec)
INFO:tensorflow:global_step/sec: 546.995
INFO:tensorflow:loss = 58075960000.0, step = 24401 (0.183 sec)
INFO:tensorflow:global_step/sec: 575.44
INFO:tensorflow:loss 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [18]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(
     X_test_new,
    shuffle=False
)

In [19]:
result = list(model.predict(pred_input_func))

INFO:tensorflow:Restoring parameters from /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpgy7lwew5/model.ckpt-25000


In [20]:
y_pred = [
    record['predictions'][0]
    for record in result
]

In [21]:
print(list(y_test[:5]))

[406200.0, 500001.0, 122900.0, 209100.0, 500001.0]


In [22]:
print(y_pred[:5])

[240488.03, 322664.8, 211969.39, 189967.17, 289343.22]


In [23]:
test_input_func = tf.estimator.inputs.pandas_input_fn(
    X_test_new,
    y_test,
    batch_size=10,
    shuffle=False
)

In [24]:
model.evaluate(test_input_func)

INFO:tensorflow:Starting evaluation at 2018-02-15-15:30:34
INFO:tensorflow:Restoring parameters from /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpgy7lwew5/model.ckpt-25000
INFO:tensorflow:Finished evaluation at 2018-02-15-15:30:35
INFO:tensorflow:Saving dict for global step 25000: average_loss = 9536110000.0, global_step = 25000, loss = 95238040000.0


{'average_loss': 9536110000.0, 'global_step': 25000, 'loss': 95238040000.0}

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test, y_pred) ** 0.5

97652.96652358791